# Transformer介绍
——Xuehuan Li

## 背景介绍
传统的序列转导模型都是基于复杂的循环或卷积神经网络实现的。在这种机制下，当前的输出取决于前一时刻的输出和当前的输入，即必须在得到前一时刻的输出后才能进行当前时刻的运算并输出。这种机制的问题就是缺少并行性，整体运算效率较低。比较好的序列转导模型一般有着encoder-decoder的模型框架，且encoder和decoder通过attention机制连接在一起。

Transformer模型于2017年由《Attention Is All You Need》一文提出，该模型沿用了encoder-decoder结构，且极大地发挥了对attention机制的使用。它仅基于attention机制，完全省掉了循环和卷积。实验表明这种方法比传统序列转导模型的训练效果好，而且它可以并行运算，提高了训练效率。

## 整体结构
Transformer模型的整体结构如下图所示，包括embedding和positional encoding、encoder(下图左侧灰底框部分)和decoder(下图右侧灰底框部分)。

<img src="./Transformer_image/transformer architecture.png" style="zoom:55%">

### Embedding和 Positional Encoding
Embedding的作用是将输入的字转为可用于计算的字向量。以翻译为例，输入汉字“我”，但模型不认识且无法计算，所以用embedding将“我”转为一个数字构成的字向量，维度为$d_{model}$。论文中定义每个字向量的维度是512。

Positional Encoding用于表示一个字在句子中的位置信息。由于Transformer是并行运算的，所以需要保留字的顺序。表示位置信息的公式为

$PE_{(pos,2i)}=\sin(pos/10000^{2i/d_{model}})$ 

$PE_{(pos,2i+1)}=\cos(pos/10000^{2i/d_{model}})$

pos表示这个字在句子中的位置，如pos=1说明该字是句子的第一个字；2i和2i+1表示字向量的维度，维度为偶数时使用sin函数，为奇数时使用cos函数。

对于整体，inputs是n个字构成的句子；经过Input Embedding后，得到n$*$d_model的矩阵；给矩阵加上Positional Encoding的位置信息，仍是nd_model的矩阵。对于上图的Outputs，Output Embedding、Positional Encoding也是一样的。示意图如下。

<img src="./Transformer_image/p e.png" style="zoom:55%">

### encoder
Transformer的编码器部分由6个encoder层组成，每个encoder层的构成如下图所示。可见：每个encoder层包含两个子层：第1个子层是多头自注意力层（Multi-Head Attention），第2个子层是前馈网络层（Feed Forward）。在每个子层之后，接着使用残差连接和正则化。

<img src="./Transformer_image/en.png" style="zoom:30%">

#### 1单头注意力机制
query是查询向量，key是键向量，value是值向量。
如下图所示，一个字向量$(1*d_{model})$和三个参数向量$(W^{Q}、W^{K}、W^{V})$分别相乘后（图中以一个输入对应3个箭头输出表示），得到自己的$q(1*d_k)、k(1*d_k)、v(1*d_v)$向量。对于某一个字向量的q，分别计算出其与其他所有k向量（包括自己的k向量）的叉乘，得到一系列的注意力值$α_{ij}$；将所有注意力值分别除以$\sqrt{d_k }$，再经过softmax，得到每个注意力值占总共注意力值的百分比（注意力分数）$α ̂{_ij}$；将每个分数和对应的v向量进行点乘，得到一系列向量b，所有向量b加起来，就得到了最后的向量$(1*d_v)$。

以第0个字为例，第0个字向量$a^0$乘以三个参数向量后得到了自己的$q^0,k^0,v^0$向量；用q^0分别乘以自己和其他的k向量，得到4个注意力值；注意力值经过softmax后得到注意力分数，用分数乘以各自对应的v向量再求和，得到的是第0个字向量与其他的字向量间的关系。

<img src="./Transformer_image/s h a.png" style="zoom:50%">

#### 2Scaled Dot-Product Attention
将注意力机制中得到q、k、v向量之后的步骤提取出来，就是scaled Dot-Product Attention。
在实际运算时，为了提高效率，会将所有的q、k、v向量分别打包成为矩阵Q、K、V。如下图所示，每个矩阵的维度都标记了出来。首先矩阵Q和矩阵K转置后的结果$K^T$进行叉乘，得到$n*n$的矩阵，然后除以$sqrt{d_k }$，(Mask是可选的，decoder部分会用到)经过softmax得到注意力分数，最后再与矩阵V相乘。

<img src="./Transformer_image/d p a.png" style="zoom:45%">

#### 3Multi-Head Attention子层
现在来宏观讲讲Transformer的encoder层的多头注意力子层。多头注意力机制可以使得模型更好地关注一个句子中不同位置的信息，其结构如下图所示。标注部分中，n是一个句子含有的字的个数；$d_{model}$是一个字向量含有的维度数量；论文中q向量和k向量的维数都是$d_k$；v向量的维数是$d_v$；h是多头注意力的“头”的数量。

<img src="./Transformer_image/m h a.png" style="zoom:45%">

上图中红色字体为输入或输出的矩阵大小，棕色框是对部分内容的解释。注意，这个图中最下方写的Q，K，V和前一部分Scaled Dot-Product Attention输入的Q、K、V是不一样的。这里的Q、K、V是最初仅经过embedding和positional encoding处理的输出，Q、K、V三者一致；而前一部分Scaled Dot-Product Attention用到的Q、K、V是对输入乘以参数矩阵后的结果，对应到该图中就是经过3个线性变换后得到的东西。

在论文中，设置的“头”的数量为8(即h=8)，并定义$d_k=d_v=d_{model}/h=64$。所以，对于输入V、K、Q，它有8组不同的$W^V,W^k,W^Q$参数矩阵。经过上图中的线性变换后，会有8组不同的更新后的V、K、Q；然后这8组V、K、Q分别经过Scaled Dot-Product Attention子层后，输出8个每个字与其他字之间关系的矩阵；接着，将这8个大小为$n*d_v$的矩阵连接起来，得到$n*(d_v*h)$大小的矩阵。不难发现对于Multi-Head Attention层来说，其输入和输出的矩阵大小是一样的，均是$d_model$，因为$d_v=d_{model}/h$。

#### 5ADD&Norm
ADD&NORM负责残差连接和正则化，其输出为LayerNorm(x+Sublayer(x))，其中Sublayer(x)由其自身实现。为了简便，其输出维度也是为$d_{model}$。

#### 4Feed Forward子层
encoder层的Feed Forward子层是一个全连接网络，输入和输出的维度均是$d_{model}=512$；还有一个维度为$d_{ff}=2048$的中间层，该层带有ReLU激活函数。Feed Forward子层的表达可写为
$$FFN(x)=max(0,xW_1+b_1)W_2+b_2$$

### decoder
Transformer的解码器部分有6个decoder层，且每个decoder层除了接收来自上一层的输出外，还会接收编码器部分最后一个encoder层的输出。示意图如下。

<img src="./Transformer_image/e d 6.png" style="zoom:35%">

decoder层的构成如下图所示。可以看到，其上面的两个子层和encoder的类似，只是其中间的 Multi-Head Attention子层的输入Q矩阵来自Masked Multi-Head Attention的输出，而K、V矩阵来自编码器部分的最后一个encoder子层。

<img src="./Transformer_image/de.png" style="zoom:40%">

最下面的Masked Multi-Head Attention子层用于隐去当前输入的后续信息。比如，输入 “I am a student”时，我们不会一次输入这一整个句子，而是一个一个词输入。在当前时刻t1将”am”传进去时，decoder会结合之前的输入“I”和当前输入“am”预测下一个单词可能是“a”。通过隐去后面的内容，确保在预测下一位置的内容时仅依赖于当前和之前的输入，避免其其预获取后续内容。

Mask的操作在前述Scaled Dot-Product Attention部分实现。其通过产生一个上三角矩阵，掩盖不该出现的后续内容。如下图所示。

<img src="./Transformer_image/em pe.png" style="zoom:50%">

### 最后的输出部分
<img src="./Transformer_image/out.png" style="zoom:70%">

最后的输出部分通过线性转导层（神经网络）和softmax函数将decoder的输出转为预测下一个字的概率。Inputs Embedding、Outputs Embedding、最后输出部分使用的权重矩阵是一样的。

## 参考文献

[1] $\mathit{Attention\;is\;all\;you\;need\;}  $https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf

[2] https://zhuanlan.zhihu.com/p/403433120

[3] https://zhuanlan.zhihu.com/p/130883313

[4] https://blog.csdn.net/longxinchen_ml/article/details/86533005

[5] https://blog.csdn.net/fs1341825137/article/details/120247499

[6] https://blog.csdn.net/u012526436/article/details/86295971